In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

# Objetivo

Você foi escalado para um time de investimentos e precisará realizar um modelo preditivo com dados da IBOVESPA para criar uma série temporal e prever diariamente o fechamento da base.

- Apresentar modelo com storytelling;

- Justificar a técnica utilizada;

- Atingir acurácia mínimia de 70%.

**Dados**: https://br.investing.com/indices/bovespa-historical-data

# Leitura e processamento dos dados

In [2]:
dados_path = './data/dados_ibovespa_2000-2023.csv'

dados = pd.read_csv(dados_path)

dados

,Data,Último,Abertura,Máxima,Mínima,Vol.,Var%
0,11.03.2021,114.984,112.782,115.127,112.776,"12,09M","1,96%"
1,10.03.2021,112.776,111.331,112.928,109.999,"12,51M","1,30%"
2,09.03.2021,111.331,110.611,112.525,109.343,"12,14M","0,65%"
3,08.03.2021,110.612,115.202,115.202,110.268,"13,70M","-3,98%"
4,05.03.2021,115.202,112.690,115.504,112.504,"13,25M","2,23%"
...,...,...,...,...,...,...,...
4994,04.01.2001,16.675,16.597,16.933,16.587,"331,23M","0,46%"
4995,03.01.2001,16.599,15.425,16.599,15.251,"381,20M","7,61%"
4996,02.01.2001,15.425,15.242,15.621,15.174,"230,63M","1,09%"
4997,28.12.2000,15.259,15.188,15.269,15.132,"324,26M","0,48%"


In [3]:
print(dados.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4999 entries, 0 to 4998
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Data      4999 non-null   object 
 1   Último    4999 non-null   float64
 2   Abertura  4999 non-null   float64
 3   Máxima    4999 non-null   float64
 4   Mínima    4999 non-null   float64
 5   Vol.      4998 non-null   object 
 6   Var%      4999 non-null   object 
dtypes: float64(4), object(3)
memory usage: 273.5+ KB
None


In [4]:
print(dados.isna().sum())

Data        0
Último      0
Abertura    0
Máxima      0
Mínima      0
Vol.        1
Var%        0
dtype: int64


Existem algumas inconsistências na base. Coluna de data está como tipo texto, variáveis numéricas em formato texto e existe um valor nulo na coluna "Vol."

## Ajuste nos tipos de dados

In [11]:
# Coluna Data com tipo datetime
dados = pd.read_csv(dados_path, parse_dates=[0])

dados.head()

,Data,Último,Abertura,Máxima,Mínima,Vol.,Var%
0,2021-11-03,114.984,112.782,115.127,112.776,"12,09M","1,96%"
1,2021-10-03,112.776,111.331,112.928,109.999,"12,51M","1,30%"
2,2021-09-03,111.331,110.611,112.525,109.343,"12,14M","0,65%"
3,2021-08-03,110.612,115.202,115.202,110.268,"13,70M","-3,98%"
4,2021-05-03,115.202,112.690,115.504,112.504,"13,25M","2,23%"


In [12]:
dados['Vol.'] = dados['Vol.'].str[:-1]
dados['Var%'] = dados['Var%'].str[:-1]

dados['Vol.'] = dados['Vol.'].str.replace(',','.')
dados['Var%'] = dados['Var%'].str.replace(',','.')

dados['Vol.'] = dados['Vol.'].astype(float)
dados['Var%'] = dados['Var%'].astype(float)

In [14]:
print(dados.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4999 entries, 0 to 4998
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   Data      4999 non-null   datetime64[ns]
 1   Último    4999 non-null   float64       
 2   Abertura  4999 non-null   float64       
 3   Máxima    4999 non-null   float64       
 4   Mínima    4999 non-null   float64       
 5   Vol.      4998 non-null   float64       
 6   Var%      4999 non-null   float64       
dtypes: datetime64[ns](1), float64(6)
memory usage: 273.5 KB
None


## Valores nulos

In [15]:
dados.loc[dados['Vol.'].isna()]

,Data,Último,Abertura,Máxima,Mínima,Vol.,Var%
1258,2016-10-02,40.377,40.592,40.592,39.96,NaN,-0.53


Por se tratar de apenas uma linha no dataframe, será adotada a técnica de exclusão da linha com valores nulos, pois não prejudicará a representação dos dados.

In [16]:
dados.dropna(inplace=True)

dados.loc[dados['Vol.'].isna()]

,Data,Último,Abertura,Máxima,Mínima,Vol.,Var%


## Renomeando colunas

Para facilitar as análises futuras e simplificar a nomenclatura das variáveis, os nomes das colunas também serão modificados

In [17]:
print(dados.columns)

Index(['Data', 'Último', 'Abertura', 'Máxima', 'Mínima', 'Vol.', 'Var%'], dtype='object')


In [18]:
dados.columns = ['ds', 'y', 'abertura', 'max', 'min', 'volume', 'variacao']

dados.head()

,ds,y,abertura,max,min,volume,variacao
0,2021-11-03,114.984,112.782,115.127,112.776,12.09,1.96
1,2021-10-03,112.776,111.331,112.928,109.999,12.51,1.30
2,2021-09-03,111.331,110.611,112.525,109.343,12.14,0.65
3,2021-08-03,110.612,115.202,115.202,110.268,13.70,-3.98
4,2021-05-03,115.202,112.690,115.504,112.504,13.25,2.23


## Exportação da base processada

In [19]:
dados.to_csv('./data/dados_ibovespa_2000-2023_processed.csv', index=False)